In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import prince

In [2]:
df = pd.read_csv('data/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2018.csv')
df.head()

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adirond,Columbia,1001000.0,146.0,Columbia Memorial Hospital,50 to 69,125,F,White,Not Span/Hispanic,...,Major,Major,Medical,"Managed Care, Unspecified",Self-Pay,NaN,NaN,True,17591.41,3570.06
1,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Extreme,Extreme,Surgical,Medicare,Medicaid,NaN,NaN,True,106190.21,78913.24
2,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,False,5600.00,4161.53
3,New York City,Kings,7001016.0,1301.0,Kings County Hospital Center,18 to 29,112,M,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,Medicaid,NaN,NaN,True,15965.54,11341.30
4,New York City,Manhattan,7002054.0,1464.0,New York-Presbyterian Hospital - Columbia Pres...,0 to 17,104,M,Multi-racial,Not Span/Hispanic,...,Major,Moderate,Medical,Medicaid,Medicaid,Self-Pay,NaN,True,36846.83,10240.02


In [3]:
# Cleaning steps
# Drop any columns with > 30% null values
df_cleaned = df.drop(['Birth Weight', 'Payment Typology 3', 'Payment Typology 2', 'CCSR Procedure Code', 'CCSR Procedure Description'], axis=1)
df_cleaned.dropna(inplace=True)
df_cleaned = df_cleaned[df_cleaned['Length of Stay'] != '120 +']
df_cleaned.head()

,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adirond,Columbia,1001000.0,146.0,Columbia Memorial Hospital,50 to 69,125,F,White,Not Span/Hispanic,...,1.0,DISEASES AND DISORDERS OF THE NERVOUS SYSTEM,3.0,Major,Major,Medical,"Managed Care, Unspecified",True,17591.41,3570.06
1,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,4.0,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,4.0,Extreme,Extreme,Surgical,Medicare,True,106190.21,78913.24
2,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,19.0,MENTAL DISEASES AND DISORDERS,2.0,Moderate,Minor,Medical,Medicaid,False,5600.00,4161.53
3,New York City,Kings,7001016.0,1301.0,Kings County Hospital Center,18 to 29,112,M,Black/African American,Not Span/Hispanic,...,5.0,DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,2.0,Moderate,Minor,Medical,Medicaid,True,15965.54,11341.30
4,New York City,Manhattan,7002054.0,1464.0,New York-Presbyterian Hospital - Columbia Pres...,0 to 17,104,M,Multi-racial,Not Span/Hispanic,...,4.0,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3.0,Major,Moderate,Medical,Medicaid,True,36846.83,10240.02


In [4]:
#Predict Length of Stay
y = df_cleaned[['Length of Stay']]
X = df_cleaned[['Age Group', 'Type of Admission', 'Patient Disposition', 'APR Severity of Illness Code', 'APR Severity of Illness Description', 'APR Risk of Mortality', 'Emergency Department Indicator']]

print(y.columns)

Index(['Length of Stay'], dtype='object')


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

categorical_columns = X_train.select_dtypes(exclude='number')
print(categorical_columns.columns)

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train[column].replace(values, encoded_values, inplace=True)

Index(['Age Group', 'Type of Admission', 'Patient Disposition',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'Emergency Department Indicator'],
      dtype='object')


In [13]:
regr = LinearRegression()
  
regr.fit(X_train, y_train)
categorical_columns = X_test.select_dtypes(exclude='number')
print(categorical_columns.columns)

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_test[column].replace(values, encoded_values, inplace=True)
preds = regr.predict(X_test)
y_test['Length of Stay'] = y_test['Length of Stay'].astype('float64')
print(y_test.dtypes)
mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

Index([], dtype='object')
Length of Stay    float64
dtype: object
The model performance for testing set
--------------------------------------
MAE is 3.3997269220605024
MSE is 43.46674767914679
R2 score is 0.161362055349218


In [18]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
dt.fit(X_train, y_train)
yhat = dt.predict(X_test)

mae = metrics.mean_absolute_error(y_test, yhat)
mse = metrics.mean_squared_error(y_test, yhat)
r2 = metrics.r2_score(y_test, yhat)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
MAE is 3.2214103741299436
MSE is 40.030473967065134
R2 score is 0.22766077050568267


In [24]:
#Try Recursive Feature Elimination
from sklearn.feature_selection import RFE

y = df_cleaned[['Length of Stay']]
X = df_cleaned.loc[:, ~df_cleaned.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]

X_train_all_features, X_test_all_features, y_train_all_features, y_test_all_features = train_test_split(X, y, test_size = 0.25)
# X_train_all_features = df_cleaned.loc[:, ~df_cleaned.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
categorical_columns = X_train_all_features.select_dtypes(exclude='number')

for column in categorical_columns:
    values = df_cleaned[column].unique()
    encoded_values = [i for i in range(1, len(values) + 1)]
    X_train_all_features[column].replace(values, encoded_values, inplace=True)
    
estimator = DecisionTreeRegressor(criterion="squared_error", max_depth=10)
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_all_features, y_train_all_features)

selector.ranking_

array([17,  6,  3, 11,  4,  8, 16, 20, 18, 19,  1,  1, 21, 10, 14,  1,  1,
        9,  7,  1,  5, 15,  2, 13, 12])

In [22]:
X_train_all_features.columns[np.where(selector.ranking_ == 1)]

Index(['Type of Admission', 'Patient Disposition', 'APR DRG Code',
       'APR DRG Description', 'APR Severity of Illness Code'],
      dtype='object')

In [17]:
from sklearn.ensemble import RandomForestRegressor

X_train_feature_selection = X_train_all_features[X_train_all_features.columns[np.where(selector.ranking_ == 1)]]
rf = RandomForestRegressor(max_depth=2, random_state=42)

rf.fit(X_train_feature_selection, y_train)

preds = rf.predict(X_test)

mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

<ipython-input-17-def5954611f7>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


The model performance for testing set
--------------------------------------
MAE is 3.4514646599429923
MSE is 44.240401312240394
R2 score is 0.14643535097468063


In [19]:
from sklearn import datasets, ensemble

params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

preds = reg.predict(X_test)

mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The model performance for testing set
--------------------------------------
MAE is 3.2510253792302644
MSE is 40.710547906132014
R2 score is 0.2145395723272593
